# Inteligência Artificial
## Aluno: Flávio Correia de Sousa Filho
## Matrícula: 397739

## Trabalho de classificação e análise
### O conjunto de dados escolhidos é um que remete a classificação de gestos musculares  do usuário para saber a ação de uma prótese. Ele vem com 4 arquivos .csv, cada arquivo tem 65 colunas, a ultima remetente a classe e as outras relacionadas a captura de um sensor.
### As quatro classes representadas são os gestos manuais pedra, papel, tesoura e ok.

### As colunas dos datasets foram nomeadas manualmente como mrXsY onde X é a leitura muscular e Y é o sensor

### Os modelos utilizados foram:
### 1- KNN
### 2- Logistic Regression
### 3- Random Forest
### 4- Adaboost
### 5- Gaussian Naive Bayes

### As métricas utilizadas foram a matriz de confusão, precision, recall, f1-score, conjunto de médias e acurácia



### Link do dataset com mais informações: https://www.kaggle.com/kyr7plus/emg-4#0.csv


In [60]:
import pandas as pd

### Lemos os quatro datasets

In [61]:
data0 = pd.read_csv("0.csv")
data1 = pd.read_csv("1.csv")
data2 = pd.read_csv("2.csv")
data3 = pd.read_csv("3.csv")

In [62]:
data0.shape, data1.shape, data2.shape, data3.shape

((2910, 65), (2903, 65), (2943, 65), (2922, 65))

### Vemos que eles não estão balanceados e isso pode prejudicar o treinamento

In [63]:
data0 = data0.iloc[:2904, :]
data2 = data2.iloc[:2904, :]
data3 = data3.iloc[:2904, :]

In [64]:
data0.shape, data1.shape, data2.shape, data3.shape

((2904, 65), (2903, 65), (2904, 65), (2904, 65))

### Agora com bases de mesmo tamanho, vamos separar os dados X0, X1, X2 e X3 das classes y0, y1, y2 e y3. 
### Depois juntamos todos os Xs e ys e realizamos a divisão em treino e teste
### Para o treino 80% e o  para o teste 20%

In [65]:
from sklearn.model_selection import train_test_split

In [66]:
X0 = data0.iloc[:, :-1]
y0 = data0.iloc[:, -1]
X1 = data1.iloc[:, :-1]
y1 = data1.iloc[:, -1]
X2 = data2.iloc[:, :-1]
y2 = data2.iloc[:, -1]
X3 = data3.iloc[:, :-1]
y3 = data3.iloc[:, -1]

X = pd.concat([X0, X1, X2, X3])
y = pd.concat([y0, y1, y2, y3])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

### Normalizando os dados para realizar a comparação com dados não normalizados

In [67]:
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

std_scale = preprocessing.StandardScaler().fit(X_train)
X_train_std = std_scale.transform(X_train)
X_test_std  = std_scale.transform(X_test)

# Usando KNN

In [68]:
from sklearn import neighbors

### Para o KNN geramos uma lista com possíveis K valores e escolheremos aquele com melhor acurácia

In [69]:
kList = [int(i) for i in range(3,20,2)]
valores = [[], []]
for kNumber in kList:
    clfKNN = neighbors.KNeighborsClassifier(kNumber, weights = 'uniform')
    trained_model = clfKNN.fit(X_train, y_train)
    valores[0] += [kNumber]
    valores[1] += [trained_model.score(X_test, y_test)]

In [70]:
bestK = 0
for i in range(1, len(valores[0])):
    if valores[1][bestK] < valores[1][i]:
        bestK = i

print("O melhor valor para K foi",valores[0][bestK],"obtendo acurácia de",valores[1][bestK])

clfKNN = neighbors.KNeighborsClassifier(valores[0][bestK], weights = 'uniform')
trained_model = clfKNN.fit(X_train, y_train)

O melhor valor para K foi 7 obtendo acurácia de 0.6840292724924666


In [71]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [72]:
predito = trained_model.predict(X_test)

In [73]:
cm = confusion_matrix(y_test, predito)
print("MATRIZ DE CONFUSÃO")
print(cm)

r = classification_report(y_test, predito)
print("\n\n"+r)

MATRIZ DE CONFUSÃO
[[527  49   8  12]
 [  0 538   3  44]
 [  0 243  99 257]
 [  3 112   3 425]]


              precision    recall  f1-score   support

           0       0.99      0.88      0.94       596
           1       0.57      0.92      0.70       585
           2       0.88      0.17      0.28       599
           3       0.58      0.78      0.66       543

    accuracy                           0.68      2323
   macro avg       0.75      0.69      0.65      2323
weighted avg       0.76      0.68      0.64      2323



### Dados normalizados 

In [74]:
valores = [[], []]
for kNumber in kList:
    clfKNN = neighbors.KNeighborsClassifier(kNumber, weights = 'uniform')
    trained_model = clfKNN.fit(X_train_std, y_train)
    valores[0] += [kNumber]
    valores[1] += [trained_model.score(X_test_std, y_test)]

In [75]:
bestK = 0
for i in range(1, len(valores[0])):
    if valores[1][bestK] < valores[1][i]:
        bestK = i

print("O melhor valor para K foi",valores[0][bestK],"obtendo acurácia de",valores[1][bestK])
clfKNN = neighbors.KNeighborsClassifier(valores[0][bestK], weights = 'uniform')
trained_model = clfKNN.fit(X_train_std, y_train)

O melhor valor para K foi 7 obtendo acurácia de 0.6577701248385708


In [76]:
predito = trained_model.predict(X_test_std)
cm = confusion_matrix(y_test, predito)
print("MATRIZ DE CONFUSÃO")
print(cm)

r = classification_report(y_test, predito)
print("\n\n"+r)

MATRIZ DE CONFUSÃO
[[371 120  37  68]
 [  0 550   0  35]
 [  3 200 190 206]
 [ 14 111   1 417]]


              precision    recall  f1-score   support

           0       0.96      0.62      0.75       596
           1       0.56      0.94      0.70       585
           2       0.83      0.32      0.46       599
           3       0.57      0.77      0.66       543

    accuracy                           0.66      2323
   macro avg       0.73      0.66      0.64      2323
weighted avg       0.74      0.66      0.64      2323



### Vemos que para o KNN a normalização melhora a matriz de confusão em geral e na detecção da classe 0, mas piora a acurácia e algumas outras métricas 

# Regressão Logística 

In [77]:
from sklearn.linear_model import LogisticRegression

In [78]:
clfLR = LogisticRegression()
modelLR = clfLR.fit(X_train, y_train)
predito = modelLR.predict(X_test)

c:\users\flavi\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\flavi\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [79]:
cm = confusion_matrix(y_test, predito)
print("MATRIZ DE CONFUSÃO")
print(cm)

r = classification_report(y_test, predito)
print("\n\n"+r)

MATRIZ DE CONFUSÃO
[[268 104 106 118]
 [ 47 194 170 174]
 [ 97 164 158 180]
 [ 94 124 126 199]]


              precision    recall  f1-score   support

           0       0.53      0.45      0.49       596
           1       0.33      0.33      0.33       585
           2       0.28      0.26      0.27       599
           3       0.30      0.37      0.33       543

    accuracy                           0.35      2323
   macro avg       0.36      0.35      0.35      2323
weighted avg       0.36      0.35      0.36      2323



### Dados normalizados 

In [80]:
clfLR = LogisticRegression()
modelLR = clfLR.fit(X_train_std, y_train)
predito = modelLR.predict(X_test_std)

c:\users\flavi\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\flavi\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [81]:
cm = confusion_matrix(y_test, predito)
print("MATRIZ DE CONFUSÃO")
print(cm)

r = classification_report(y_test, predito)
print("\n\n"+r)

MATRIZ DE CONFUSÃO
[[268 104 106 118]
 [ 47 194 169 175]
 [ 97 163 159 180]
 [ 94 124 126 199]]


              precision    recall  f1-score   support

           0       0.53      0.45      0.49       596
           1       0.33      0.33      0.33       585
           2       0.28      0.27      0.27       599
           3       0.30      0.37      0.33       543

    accuracy                           0.35      2323
   macro avg       0.36      0.35      0.35      2323
weighted avg       0.36      0.35      0.36      2323



### Não houve diferença entre dados normalizados e não, ambos apresentaram resultados não satisfatórios

# Random Forest

In [82]:
from sklearn.ensemble import RandomForestClassifier

### Usando entropy como critério

In [83]:
clf = RandomForestClassifier(criterion='entropy', random_state=0)
model = clf.fit(X_train, y_train)
predito = model.predict(X_test)

c:\users\flavi\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [84]:
cm = confusion_matrix(y_test, predito)
print("MATRIZ DE CONFUSÃO")
print(cm)

r = classification_report(y_test, predito)
print("\n\n"+r)

MATRIZ DE CONFUSÃO
[[578   0  13   5]
 [  0 526  20  39]
 [ 16  15 532  36]
 [ 25  32  35 451]]


              precision    recall  f1-score   support

           0       0.93      0.97      0.95       596
           1       0.92      0.90      0.91       585
           2       0.89      0.89      0.89       599
           3       0.85      0.83      0.84       543

    accuracy                           0.90      2323
   macro avg       0.90      0.90      0.90      2323
weighted avg       0.90      0.90      0.90      2323



### Mudando o critério para gini

In [85]:
clf = RandomForestClassifier(criterion='gini', random_state=0)
model = clf.fit(X_train, y_train)
predito = model.predict(X_test)

c:\users\flavi\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [86]:
cm = confusion_matrix(y_test, predito)
print("MATRIZ DE CONFUSÃO")
print(cm)

r = classification_report(y_test, predito)
print("\n\n"+r)

MATRIZ DE CONFUSÃO
[[574   0  13   9]
 [  0 524  19  42]
 [ 15  23 536  25]
 [ 33  33  30 447]]


              precision    recall  f1-score   support

           0       0.92      0.96      0.94       596
           1       0.90      0.90      0.90       585
           2       0.90      0.89      0.90       599
           3       0.85      0.82      0.84       543

    accuracy                           0.90      2323
   macro avg       0.89      0.89      0.89      2323
weighted avg       0.90      0.90      0.90      2323



### Vemos que o random forest tem um desempenho muito bom em relação aos outros modelos, e que tendo gini como critério obtemos melhores resultados na maioria das métricas

# Adaboost

In [87]:
from sklearn.ensemble import AdaBoostClassifier

### Criamos duas listas com possíveis parâmetros do modelo e escolheremos o melhor

In [88]:
lrList = [0.5 , 1 , 2]
nestList = [50, 100, 150]
valores = [[], []]
for lrEl in lrList:
    for nestEl in nestList:
        clf = AdaBoostClassifier(n_estimators=nestEl, learning_rate=lrEl, random_state=0)
        model = clf.fit(X_train, y_train)
        valores[0] += [(lrEl, nestEl)]
        valores[1] += [model.score(X_test, y_test)]

In [89]:
valores

[[(0.5, 50),
  (0.5, 100),
  (0.5, 150),
  (1, 50),
  (1, 100),
  (1, 150),
  (2, 50),
  (2, 100),
  (2, 150)],
 [0.8024106758501938,
  0.8519156263452432,
  0.8738699956952217,
  0.8161859664227292,
  0.8527765820060267,
  0.8885062419285407,
  0.5458458889367198,
  0.5458458889367198,
  0.5458458889367198]]

In [90]:
bestP = 0
for i in range(1, len(valores[1])):
    if valores[1][bestK] < valores[1][i]:
        bestK = i

print("O melhor valor para o estimador foi",valores[0][bestP][1],"e para o learning rate foi",valores[0][bestP][0])
clf = AdaBoostClassifier(n_estimators=valores[0][bestP][1], learning_rate=valores[0][bestP][0], random_state=0)
model = clf.fit(X_train, y_train)
predito = model.predict(X_test)

O melhor valor para o estimador foi 50 e para o learning rate foi 0.5


In [91]:
cm = confusion_matrix(y_test, predito)
print("MATRIZ DE CONFUSÃO")
print(cm)

r = classification_report(y_test, predito)
print("\n\n"+r)

MATRIZ DE CONFUSÃO
[[383   0  77 136]
 [  0 537  13  35]
 [  0  30 531  38]
 [ 11  98  21 413]]


              precision    recall  f1-score   support

           0       0.97      0.64      0.77       596
           1       0.81      0.92      0.86       585
           2       0.83      0.89      0.86       599
           3       0.66      0.76      0.71       543

    accuracy                           0.80      2323
   macro avg       0.82      0.80      0.80      2323
weighted avg       0.82      0.80      0.80      2323



### Vemos que o Adabost tem um desempenho satisfatório em relação às métricas

# Gaussian Naive Bayes

In [92]:
from sklearn.naive_bayes import GaussianNB

In [93]:
clf = GaussianNB()
model = clf.fit(X_train, y_train)
predito = model.predict(X_test)

In [94]:
cm = confusion_matrix(y_test, predito)
print("MATRIZ DE CONFUSÃO")
print(cm)

r = classification_report(y_test, predito)
print("\n\n"+r)

MATRIZ DE CONFUSÃO
[[549   0   4  43]
 [  0 556  14  15]
 [  5  14 553  27]
 [ 39  81   8 415]]


              precision    recall  f1-score   support

           0       0.93      0.92      0.92       596
           1       0.85      0.95      0.90       585
           2       0.96      0.92      0.94       599
           3       0.83      0.76      0.80       543

    accuracy                           0.89      2323
   macro avg       0.89      0.89      0.89      2323
weighted avg       0.89      0.89      0.89      2323



### Vemos que o GNB possui um rendimento satisfatório

### Analisando esse problema e comparando os modelos utilizados, podemos afirmar que o melhor modelo foi o  Random Forest 
http://localhost:8888/notebooks/trabalhoClassificacao.ipynb#Random-Forest
### E o pior foi de Regressão Logística.
http://localhost:8888/notebooks/trabalhoClassificacao.ipynb#Regress%C3%A3o-Log%C3%ADstica